In [2]:
from typing import List, Optional, Union
import pandas as pd


class properties_of:
    def __init__(self, name: str, engine: str = "pandas"):
        """
        Inicializuojama klasė. Palaikomi du varikliai: 'pandas' ir 'pyspark'.

        :param name: Objektų grupės pavadinimas.
        :param engine: Variklis ('pandas' arba 'pyspark').
        """
        self.name = name
        self.engine = engine

        if engine == "pandas":
            self.df_property = pd.DataFrame(columns=["id", "property_id", "value"])
            self.df_property_type = pd.DataFrame(columns=["property_id", "description"])
        elif engine == "pyspark":
            from pyspark.sql import SparkSession
            self.spark = SparkSession.builder.master("local").appName("PropertiesDB").getOrCreate()
            self.df_property = self.spark.createDataFrame([], schema="id STRING, property_id STRING, value STRING")
            self.df_property_type = self.spark.createDataFrame([], schema="property_id STRING, description STRING")
        else:
            raise ValueError("Nepalaikomas variklis: pasirinkite 'pandas' arba 'pyspark'.")

    def add_property(self, id: str, property_id: str, value: str, check_property_type: bool = False) -> None:
        """
        Pridedama savybė konkrečiam objektui.

        :param id: Objekto ID.
        :param property_id: Savybės ID.
        :param value: Savybės reikšmė.
        :param check_property_type: Tikrinti, ar savybės tipas egzistuoja.
        """
        if self.engine == "pandas":
            if check_property_type and property_id not in self.df_property_type["property_id"].values:
                raise ValueError(f"Savybės ID '{property_id}' nėra savybių tipų lentelėje.")
            self.df_property = pd.concat([
                self.df_property,
                pd.DataFrame({"id": [id], "property_id": [property_id], "value": [value]})
            ], ignore_index=True).drop_duplicates(subset=["id", "property_id"])
        elif self.engine == "pyspark":
            if check_property_type:
                existing = self.df_property_type.filter(f"property_id = '{property_id}'").count() > 0
                if not existing:
                    raise ValueError(f"Savybės ID '{property_id}' nėra savybių tipų lentelėje.")
            new_row = self.spark.createDataFrame([(id, property_id, value)], schema="id STRING, property_id STRING, value STRING")
            self.df_property = self.df_property.union(new_row)

    def export_to_narrow(self) -> pd.DataFrame:
        """
        Eksportuoja savybes į siauro formato Pandas DataFrame.

        :return: Pandas DataFrame su stulpeliais ['id', 'property_id', 'value'].
        """
        if self.engine == "pandas":
            return self.df_property.copy()
        elif self.engine == "pyspark":
            return self.df_property.toPandas()

    def import_from_narrow(self, df: pd.DataFrame) -> None:
        """
        Importuoja siauro formato Pandas DataFrame į duomenų bazę.

        :param df: Pandas DataFrame su stulpeliais ['id', 'property_id', 'value'].
        """
        for _, row in df.iterrows():
            self.add_property(row["id"], row["property_id"], row["value"])

    def export_to_wide(self, properties: Optional[List[str]] = None) -> pd.DataFrame:
        """
        Eksportuoja savybes į platų Pandas DataFrame.

        :param properties: Savybių sąrašas. Jei `None`, grąžina visas savybes.
        :return: Pandas DataFrame su kiekvienai savybei atskiru stulpeliu.
        """
        df_narrow = self.export_to_narrow()
        df = df_narrow.pivot(index="id", columns="property_id", values="value").reset_index()

        if properties:
            all_columns = ["id"] + properties
            df = df.reindex(columns=all_columns, fill_value=None)

        return df

    def import_from_wide(self, df: pd.DataFrame) -> None:
        """
        Importuoja savybes iš plataus Pandas DataFrame.

        :param df: Pandas DataFrame su stulpeliais: `id`, savybės ir jų reikšmės.
        """
        id_col = "id"
        for _, row in df.iterrows():
            for property_id, value in row.drop(labels=[id_col]).items():
                if pd.notna(value):
                    self.add_property(row[id_col], property_id, value)

    def close(self) -> None:
        """Uždaromas PySpark sesija, jei naudojama."""
        if self.engine == "pyspark":
            self.spark.stop()

In [3]:
def main() -> None:
    # Testavimas
    obj = properties_of("knyga", engine="pandas")

    # Test import_from_wide
    data = pd.DataFrame({
        "id": ["111-222-333", "222-333-444"],
        "title": ["Lapė Snapė", "Vilkas Pilkas"],
        "author": ["Jojas Papievis", "Antanas Antanaitis"],
        "year": ["2020", "1900"]
    })
    obj.import_from_wide(data)

    # Eksportavimas į siaurą lentelę
    narrow = obj.export_to_narrow()
    print("Siauras formatas:")
    print(narrow)

    # Importavimas iš siauro formato
    obj.import_from_narrow(narrow)

    # Eksportas į platų formatą
    print("Platus formatas:")
    print(obj.export_to_wide())

    obj.close()


if __name__ == "__main__":
    main()


Siauras formatas:
            id property_id               value
0  111-222-333       title          Lapė Snapė
1  111-222-333      author      Jojas Papievis
2  111-222-333        year                2020
3  222-333-444       title       Vilkas Pilkas
4  222-333-444      author  Antanas Antanaitis
5  222-333-444        year                1900
Platus formatas:
property_id           id              author          title  year
0            111-222-333      Jojas Papievis     Lapė Snapė  2020
1            222-333-444  Antanas Antanaitis  Vilkas Pilkas  1900
